In [2]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

d:\Crowd-Funding\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [30]:
MODEL = 'llama3.2'
db_name = 'vector_db'

In [5]:
folders = glob.glob('knowledge_base/*')
text_loader_kwargs = {'encoding': 'utf8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob='**/*.md', loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [6]:
print(documents[0])

page_content='# Admin Guidelines for Case Approval

## Admin Responsibilities

### Primary Duties
- Review campaign submissions thoroughly
- Verify all documentation authenticity
- Conduct background checks when necessary
- Communicate with campaign creators professionally
- Make fair, unbiased approval decisions
- Document reasoning for all decisions
- Escalate complex cases appropriately

### Access Levels

#### Junior Admin
- Review standard campaigns (<PKR 500,000)
- Request additional documentation
- Recommend approval/rejection (requires senior approval)
- Handle basic support queries

#### Senior Admin
- Final approval authority
- Review complex/high-value campaigns
- Handle escalated cases
- Review appeals
- Senior admin makes final decision
- Mentor junior admins
- Quality assurance checks

#### Super Admin
- System-wide oversight
- Policy decisions
- Resolve admin conflicts
- Access to all campaigns and data
- Platform configuration

## Review Process Guidelines

### Initial 

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [11]:
from langchain_community.embeddings import OllamaEmbeddings

In [13]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [14]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [16]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

In [24]:
from langchain_ollama import OllamaLLM

In [45]:
system_prompt = """You are an AI assistant for a crowdfunding platform founded by Saad Zaidi, Waliuddin Ahmed, and Sajjad Ahmed - Computer Science students at FAST University.

YOUR ROLE:
You help users understand and navigate our crowdfunding platform where people can donate to verified causes through multiple payment methods including credit/debit cards, PayPal, EasyPaisa, and JazzCash.

HOW TO RESPOND:

1. FOR GREETINGS & CASUAL MESSAGES:
   - Respond warmly and naturally to greetings (hi, hello, hey)
   - Acknowledge thanks gracefully
   - Be friendly when users say goodbye
   - Keep casual responses brief and welcoming

2. FOR QUESTIONS ABOUT THE PLATFORM:
   - Use the knowledge base context provided to answer accurately
   - Be specific and cite relevant information
   - If the knowledge base has the answer, use it confidently
   - Break down complex processes into clear steps

3. WHEN YOU'RE UNCERTAIN:
   - If the knowledge base doesn't contain the information, clearly state: "I don't have that specific information in my knowledge base"
   - Offer to help with related questions you can answer
   - Never make up information about policies, fees, or processes

4. TONE & STYLE:
   - Professional yet approachable
   - Concise but complete - don't leave out important details
   - Use simple language, avoid jargon
   - Be empathetic, especially for campaigns related to hardships
   - Stay positive and encouraging

5. KEY TOPICS YOU HANDLE:
   - Creating and managing campaigns
   - Making donations and payment methods
   - Campaign approval and verification process
   - Platform policies and security
   - Founder information and platform background
   - Troubleshooting common issues

REMEMBER: Your goal is to make users feel supported and confident using our platform, whether they're creating a campaign or making a donation."""

In [46]:
llm = OllamaLLM(model=MODEL, temprature=0.7, system=system_prompt)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [49]:
small_talk = {
    "hi": "Hello! How can I help you today?",
    "hello": "Hi there 👋",
    "thanks": "You're welcome!",
    "thank you": "Happy to help!"
}

In [50]:
def chat(message, history):
    if message.lower() in small_talk:
        return small_talk[message]
    else:
        result = conversation_chain.invoke({"question": message})
        return result['answer']

In [51]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
